<a href="https://colab.research.google.com/github/jorgemar723/3DObjectReconstruction/blob/main/notebooks/web_viewer_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
!pip install trimesh
import trimesh
from google.colab import files

print("Libraries imported successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 736.5/736.5 kB 18.9 MB/s eta 0:00:00
Libraries imported successfully!


In [ ]:
# Upload the PLY file from notebook 1
print("Please upload your reconstruction PLY file:")
uploaded = files.upload()

ply_filename = list(uploaded.keys())[0]
print(f" Loaded: {ply_filename}")


Please upload your reconstruction PLY file:


Saving chair_reconstruction.ply to chair_reconstruction.ply
 Loaded: chair_reconstruction.ply


In [ ]:
# Load the point cloud
mesh = trimesh.load(ply_filename)

# Extract vertices
if hasattr(mesh, 'vertices'):
    points = mesh.vertices
else:
    points = mesh.points  # for PointCloud objects

print(f" Loaded {len(points)} points from PLY")

# Convert to JSON format for Three.js
points_list = points.tolist()

json_data = {
    "points": points_list,
    "numPoints": len(points_list)
}

# Save to JSON file
json_filename = ply_filename.replace('.ply', '.json')
with open(json_filename, 'w') as f:
    json.dump(json_data, f)

print(f" Converted to: {json_filename}")


 Loaded 6 points from PLY
 Converted to: chair_reconstruction.json


In [ ]:
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>3D Point Cloud Viewer</title>
    <style>
        body {
            margin: 0;
            overflow: hidden;
            font-family: Arial, sans-serif;
            background: #1a1a2e;
        }
        #container {
            width: 100vw;
            height: 100vh;
        }
        #info {
            position: absolute;
            top: 20px;
            left: 20px;
            color: white;
            background: rgba(0, 0, 0, 0.7);
            padding: 15px;
            border-radius: 8px;
            font-size: 14px;
        }
        #controls {
            position: absolute;
            bottom: 20px;
            left: 20px;
            color: white;
            background: rgba(0, 0, 0, 0.7);
            padding: 15px;
            border-radius: 8px;
            font-size: 12px;
        }
    </style>
</head>
<body>
    <div id="container"></div>
    <div id="info">
        <h2>3D Reconstruction Viewer</h2>
        <p id="pointCount">Loading point cloud...</p>
    </div>
    <div id="controls">
        <strong>Controls:</strong><br>
         Left Click + Drag: Rotate<br>
        Right Click + Drag: Pan<br>
        Scroll: Zoom
    </div>

    <script src="https://cdnjs.cloudflare.com/ajax/libs/three.js/r128/three.min.js"></script>
    <script>
        let scene, camera, renderer, points, controls;

        // Load the point cloud data
        fetch('PLACEHOLDER_JSON_FILENAME')
            .then(response => response.json())
            .then(data => {
                document.getElementById('pointCount').textContent =
                    `Points: ${data.numPoints.toLocaleString()}`;
                initScene(data.points);
            })
            .catch(error => {
                console.error('Error loading point cloud:', error);
                document.getElementById('pointCount').textContent = 'Error loading point cloud';
            });

        function initScene(pointsData) {
            // Scene setup
            scene = new THREE.Scene();
            scene.background = new THREE.Color(0x1a1a2e);

            // Camera setup
            camera = new THREE.PerspectiveCamera(
                75,
                window.innerWidth / window.innerHeight,
                0.1,
                1000
            );
            camera.position.set(2, 2, 2);
            camera.lookAt(0, 0, 0);

            // Renderer setup
            renderer = new THREE.WebGLRenderer({ antialias: true });
            renderer.setSize(window.innerWidth, window.innerHeight);
            document.getElementById('container').appendChild(renderer.domElement);

            // Create point cloud geometry
            const geometry = new THREE.BufferGeometry();
            const vertices = new Float32Array(pointsData.flat());
            geometry.setAttribute('position', new THREE.BufferAttribute(vertices, 3));

            // Point cloud material
            const material = new THREE.PointsMaterial({
                size: 0.02,
                color: 0x00d4ff,
                sizeAttenuation: true
            });

            // Create point cloud mesh
            points = new THREE.Points(geometry, material);
            scene.add(points);

            // Add ambient light
            const ambientLight = new THREE.AmbientLight(0xffffff, 0.6);
            scene.add(ambientLight);

            // Add directional light
            const directionalLight = new THREE.DirectionalLight(0xffffff, 0.4);
            directionalLight.position.set(5, 5, 5);
            scene.add(directionalLight);

            // Add grid helper
            const gridHelper = new THREE.GridHelper(10, 10, 0x444444, 0x222222);
            scene.add(gridHelper);

            // Add axes helper
            const axesHelper = new THREE.AxesHelper(1);
            scene.add(axesHelper);

            // Simple orbit controls (mouse-based)
            setupControls();

            // Handle window resize
            window.addEventListener('resize', onWindowResize);

            // Start animation loop
            animate();
        }

        function setupControls() {
            let isDragging = false;
            let previousMousePosition = { x: 0, y: 0 };
            let rotation = { x: 0, y: 0 };

            renderer.domElement.addEventListener('mousedown', (e) => {
                isDragging = true;
                previousMousePosition = { x: e.clientX, y: e.clientY };
            });

            renderer.domElement.addEventListener('mousemove', (e) => {
                if (isDragging) {
                    const deltaX = e.clientX - previousMousePosition.x;
                    const deltaY = e.clientY - previousMousePosition.y;

                    rotation.y += deltaX * 0.005;
                    rotation.x += deltaY * 0.005;

                    previousMousePosition = { x: e.clientX, y: e.clientY };
                }
            });

            renderer.domElement.addEventListener('mouseup', () => {
                isDragging = false;
            });

            renderer.domElement.addEventListener('wheel', (e) => {
                e.preventDefault();
                const zoomSpeed = 0.1;
                camera.position.z += e.deltaY * 0.001 * zoomSpeed;
                camera.position.z = Math.max(0.5, Math.min(10, camera.position.z));
            });

            // Apply rotation to camera
            function updateCamera() {
                const distance = camera.position.length();
                camera.position.x = distance * Math.sin(rotation.y) * Math.cos(rotation.x);
                camera.position.y = distance * Math.sin(rotation.x);
                camera.position.z = distance * Math.cos(rotation.y) * Math.cos(rotation.x);
                camera.lookAt(0, 0, 0);
            }

            // Update in animation loop
            window.updateCameraRotation = updateCamera;
        }

        function onWindowResize() {
            camera.aspect = window.innerWidth / window.innerHeight;
            camera.updateProjectionMatrix();
            renderer.setSize(window.innerWidth, window.innerHeight);
        }

        function animate() {
            requestAnimationFrame(animate);
            if (window.updateCameraRotation) window.updateCameraRotation();
            renderer.render(scene, camera);
        }
    </script>
</body>
</html>
"""

# Replace placeholder with actual JSON filename
html_content = html_content.replace('PLACEHOLDER_JSON_FILENAME', json_filename)

# Save HTML file
html_filename = 'point_cloud_viewer.html'
with open(html_filename, 'w') as f:
    f.write(html_content)

print(f" Created: {html_filename}")


 Created: point_cloud_viewer.html


In [24]:
# Download both JSON and HTML
print("Downloading files...")

files.download(json_filename)
print(f"Downloaded: {json_filename}")

files.download(html_filename)
print(f"Downloaded: {html_filename}")

print("\n" + "="*50)
print("INSTRUCTIONS:")
print("="*50)
print("1. Save both files to the same folder on your computer")
print("2. Open 'point_cloud_viewer.html' in a web browser")
print("3. Use mouse to rotate, pan, and zoom the 3D point cloud")
print("="*50)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: real_object_reconstruction.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded: point_cloud_viewer.html

INSTRUCTIONS:
1. Save both files to the same folder on your computer
2. Open 'point_cloud_viewer.html' in a web browser
3. Use mouse to rotate, pan, and zoom the 3D point cloud


In [ ]:
print("\n✓ Web viewer setup complete!")
print(f"\nGenerated files:")
print(f"  - {json_filename} (point cloud data)")
print(f"  - {html_filename} (interactive viewer)")
print(f"\nTo view: Open {html_filename} in Chrome, Firefox, or Edge")


✓ Web viewer setup complete!

Generated files:
  - chair_reconstruction.json (point cloud data)
  - point_cloud_viewer.html (interactive viewer)

To view: Open point_cloud_viewer.html in Chrome, Firefox, or Edge
